In [ ]:
# Install Optuna for hyperparameter optimization
!pip install --quiet optuna optuna-integration

In [ ]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

drive_basepath = '/content/drive/MyDrive/ML/Project'
!unzip -nq '{drive_basepath}/test_data_unlabeled.zip'
!unzip -nq '{drive_basepath}/train_data_unlabeled.zip'
!cp -n '{drive_basepath}/y_train.csv' .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import keras
import re
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, AlphaDropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from sklearn.model_selection import train_test_split
import optuna
from optuna.integration import KerasPruningCallback
from optuna.integration import TFKerasPruningCallback
import zipfile
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
x_train_path = "train_data_unlabeled"
y_train_path = "y_train.csv"
x_test_path = "test_data_unlabeled"

In [ ]:
# Function for loading test data and preprocessing
def load_train_data(images_path, csv_path):
    # Read the CSV file
    df = pd.read_csv(csv_path)

    # Load images and preprocess
    images = []
    labels = []

    for _, row in df.iterrows():
        file = os.path.join(images_path, f"img_{row['id'] + 1}.png")
        if not os.path.exists(file):
            print(f"Image file {file} not found. Skipping.")
            continue

        try:
            # Open the image
            image = Image.open(file)

            # Convert to RGB
            image = image.convert('RGB')

            # Crop the image by 2 pixels from each side, which removes white border
            width, height = image.size
            left, upper, right, lower = 2, 2, width - 2, height - 2
            image = image.crop((left, upper, right, lower))

            # Resize to the actual amount of pixels
            image = image.resize((50, 44))

            # Normalize pixel values
            image = np.array(image).astype('float32') / 255.0

            images.append(image)
            labels.append(row['target'])

        except IOError as e:
            print(f"Error reading image {file}: {e}")
            continue

    images = np.array(images)
    labels = pd.get_dummies(labels)

    return images, labels

In [ ]:
x_train, y_train = load_train_data(x_train_path, y_train_path)

In [ ]:
# Check number of samples
print(f"Number of samples in x_train: {len(x_train)}")
print(f"Number of samples in y_train: {len(y_train)}")

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

Number of samples in x_train: 16182
Number of samples in y_train: 16182


In [ ]:
# Define the objective function for Optuna
def objective(trial):
    # Hyperparameters to be tuned
    conv2d_filter = trial.suggest_categorical('conv2d_filter', [32, 64, 128])
    dense_units = trial.suggest_categorical('dense_units', [256, 512, 1024])
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.7)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)

    # Model creation using the suggested hyperparameters
    model = Sequential()
    model.add(Conv2D(conv2d_filter, (3, 3), activation='relu', input_shape=(44, 50, 3)))
    model.add(Conv2D(conv2d_filter, (3, 3), activation='relu'))
    model.add(Conv2D(conv2d_filter, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, activation='softmax'))

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, restore_best_weights=True)

    # KerasPruningCallback
    pruning_callback = KerasPruningCallback(trial, 'val_accuracy')

    # Fit the model
    history = model.fit(
        x_train,
        y_train,
        batch_size=64,
        epochs=25,  # Modify
        callbacks=[early_stopping, pruning_callback],
        validation_data=(x_val, y_val),
        verbose=0
    )

    # Evaluate the model on the validation set
    score = model.evaluate(x_val, y_val, verbose=1)
    return score[1]  # Return the accuracy

In [ ]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=25, timeout=600)

# Print the best hyperparameters
print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

print(f'Value: {trial.value}')
print('Params: ')
for key, value in trial.params.items():
    print(f'{key}: {value}')

[I 2024-04-20 17:03:44,289] A new study created in memory with name: no-name-6217f882-0352-4e84-80b2-c4709ad3edf0


Restoring model weights from the end of the best epoch: 18.
Epoch 23: early stopping
102/102 [==============================] - 1s 6ms/step - loss: 0.1038 - accuracy: 0.9694


[I 2024-04-20 17:06:07,793] Trial 0 finished with value: 0.9694161415100098 and parameters: {'conv2d_filter': 128, 'dense_units': 512, 'dropout_rate': 0.23670986176646383, 'learning_rate': 0.00011908830753699454}. Best is trial 0 with value: 0.9694161415100098.


102/102 [==============================] - 0s 3ms/step - loss: 0.1624 - accuracy: 0.9404


[I 2024-04-20 17:06:53,670] Trial 1 finished with value: 0.940376877784729 and parameters: {'conv2d_filter': 32, 'dense_units': 1024, 'dropout_rate': 0.28452826768414524, 'learning_rate': 3.586992374289203e-05}. Best is trial 0 with value: 0.9694161415100098.


...

Restoring model weights from the end of the best epoch: 20.
Epoch 25: early stopping
102/102 [==============================] - 0s 4ms/step - loss: 0.1615 - accuracy: 0.9642


[I 2024-04-20 17:13:13,758] Trial 6 finished with value: 0.9641643762588501 and parameters: {'conv2d_filter': 64, 'dense_units': 1024, 'dropout_rate': 0.18547616007204878, 'learning_rate': 0.0035717641711660755}. Best is trial 0 with value: 0.9694161415100098.


Restoring model weights from the end of the best epoch: 9.
Epoch 14: early stopping
102/102 [==============================] - 1s 5ms/step - loss: 0.1221 - accuracy: 0.9605


[I 2024-04-20 17:14:29,390] Trial 7 finished with value: 0.9604572057723999 and parameters: {'conv2d_filter': 128, 'dense_units': 256, 'dropout_rate': 0.2591805413932094, 'learning_rate': 0.002956944278122597}. Best is trial 0 with value: 0.9694161415100098.


Number of finished trials: 8
Best trial:
Value: 0.9694161415100098
Params: 
conv2d_filter: 128
dense_units: 512
dropout_rate: 0.23670986176646383
learning_rate: 0.00011908830753699454


In [ ]:
# Retrieve the best hyperparameters from the Optuna study
best_conv2d_filter = study.best_params['conv2d_filter']
best_dense_units = study.best_params['dense_units']
best_dropout_rate = study.best_params['dropout_rate']
best_learning_rate = study.best_params['learning_rate']

# Create the final model using the best hyperparameters
model = Sequential()
model.add(Conv2D(best_conv2d_filter, (3, 3), activation='relu', input_shape=(44, 50, 3)))
model.add(Conv2D(best_conv2d_filter, (3, 3), activation='relu'))
model.add(Conv2D(best_conv2d_filter, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(best_dense_units, activation='relu'))
model.add(Dropout(best_dropout_rate))
model.add(Dense(4, activation='softmax'))

# Compile the final model
optimizer = Adam(learning_rate=best_learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, restore_best_weights=True)
pruning_callback = KerasPruningCallback(trial, 'val_accuracy')

# Train the model on the entire training data
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=50,  # Adjust as needed
    callbacks=[early_stopping, pruning_callback],
    validation_data=(x_val, y_val),
    verbose=1
)

# Evaluate the final model on the validation set
final_score = model.evaluate(x_val, y_val, verbose=1)
print("Final validation accuracy:", final_score[1])

# Save the trained model
model.save('final_top_model.keras')

Epoch 1/50
  5/203 [..............................] - ETA: 5s - loss: 1.3528 - accuracy: 0.2812

203/203 [==============================] - 8s 33ms/step - loss: 0.8306 - accuracy: 0.6535 - val_loss: 0.5217 - val_accuracy: 0.7998
Epoch 2/50
203/203 [==============================] - 6s 31ms/step - loss: 0.4496 - accuracy: 0.8253 - val_loss: 0.3564 - val_accuracy: 0.8758
Epoch 3/50
203/203 [==============================] - 6s 32ms/step - loss: 0.3237 - accuracy: 0.8736 - val_loss: 0.2570 - val_accuracy: 0.9055
Epoch 4/50
203/203 [==============================] - 6s 31ms/step - loss: 0.2601 - accuracy: 0.9008 - val_loss: 0.2244 - val_accuracy: 0.9123
Epoch 5/50
203/203 [==============================] - 6s 32ms/step - loss: 0.2012 - accuracy: 0.9245 - val_loss: 0.1930 - val_accuracy: 0.9277
Epoch 6/50
203/203 [==============================] - 6s 31ms/step - loss: 0.1683 - accuracy: 0.9380 - val_loss: 0.1613 - val_accuracy: 0.9388
Epoch 7/50
203/203 [==============================] - 6s 32ms/step - loss: 0.1364 - accuracy: 0.9509 - val_loss: 0.1508 - val_accuracy: 0.9422
Epoch 8/50

In [ ]:
# Sorting function that sorts strings numerically when they contain numbers
def natural_sort_key(file_name):
    return [int(s) if s.isdigit() else s.lower() for s in re.split('(\d+)', file_name)]

# Function to load and preprocess test data
def load_test_data(images_path):
    images = []

    # Load PNG images
    all_files = os.listdir(images_path)
    png_files = [file for file in all_files if file.endswith('.png')]

    # Sort images by filename using natural sort order
    file_names = sorted(png_files, key=natural_sort_key)

    for file_name in file_names:
        file = os.path.join(images_path, file_name)
        try:
            # Open the image
            image = Image.open(file)

            # Convert to RGB
            image = image.convert('RGB')

            # Crop the image by 2 pixels from each side, which removes white border
            width, height = image.size
            left, upper, right, lower = 2, 2, width - 2, height - 2
            image = image.crop((left, upper, right, lower))

            # Resize to actual amount of pixels
            image = image.resize((50, 44))

            # Normalize pixel values
            image = np.array(image).astype('float32') / 255.0

            images.append(image)

        except IOError as e:
            print(f"Error reading image {file}: {e}")
            continue

    images = np.array(images)
    return images

In [ ]:
# Load model
saved_model = load_model('final_top_model.keras')

# Load test data
x_test = load_test_data(x_test_path)

# Generate output predictions y_test from x_test
y_test_pred = saved_model.predict(x_test)

150/150 [==============================] - 1s 7ms/step


In [ ]:
# Apply a probability threshold
y_test_labels = np.argmax(y_test_pred, axis=1)

print(y_test_pred)
print(y_test_labels)

# Create a CSV file in the right format for Kaggle submission
y_test = pd.DataFrame({'id': range(len(y_test_labels)), 'target': y_test_labels})
y_test.to_csv('y_test.csv', index=False)

[[5.37341449e-16 9.99976039e-01 2.39582350e-05 2.51419856e-15]
 [6.80670135e-30 3.53805838e-08 1.76447923e-08 1.00000000e+00]
 [5.97472995e-33 1.04375344e-16 3.35356500e-11 1.00000000e+00]
 ...
 [1.00000000e+00 2.38144245e-15 7.41090555e-15 4.84907219e-14]
 [1.00000000e+00 5.79723378e-16 3.05050403e-15 1.10119721e-14]
 [1.26533202e-08 1.40807490e-04 9.99859214e-01 1.24525634e-08]]
[1 3 3 ... 0 0 2]


In [ ]:
!cp y_test.csv '{drive_basepath}'
!cp final_top_model.keras '{drive_basepath}'

as